In [44]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [45]:
df = pd.read_csv('train.csv')
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


<h2 style='color:yellow;'>Step1 : Train / Test / Split</h2>

In [46]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [47]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [48]:
## Since our Dataset contains missing values we have to fix them out

In [49]:
#Applying Imputation

si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.fit_transform(x_test[['Age']])
x_test_embarked = si_embarked.fit_transform(x_test[['Embarked']])

In [50]:
## Now Dataset contains two categorical columns which is Sex and Embarked

In [51]:
#One Hot Encoding for Sex and Embarked

In [52]:
ohe_sex = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

x_test_sex = ohe_sex.fit_transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)

In [53]:
x_test_embarked.shape

(179, 3)

In [54]:
x_train_rem = x_train.drop(columns=['Age','Sex','Embarked'])
x_test_rem = x_test.drop(columns=['Age','Sex','Embarked'])

In [35]:
## now we have seperate arrays of age,embarked(missing), sex,embarked
## and now we will combine them

In [56]:
x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis = 1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis = 1)

In [60]:
x_train_transformed.shape
x_test_transformed.shape

(179, 10)

<h2 style='color:yellow;'>Applying Decision Tree (Model)</h2>

In [61]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [62]:
y_pred = clf.predict(x_test_transformed)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7541899441340782

In [64]:
## NOW IF WE WANT THIS MODEL TO DEPLOY IN THE WEBSITE WHERE IF USER ENTERS THE GIVEN INPUTS AND THE MODEL PREDICTS SURIVIVED OR NOT
##WE HAVE TO EXPORT 

In [65]:
import pickle

In [67]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))